# Solving Travelling salesman using dynamic programming

* author: Eole Cervenka
* date: 04/17/2016

### Premise
I started this project to optimize the operations of the organization [**Lovin' Spoonful**](http://lovinspoonfulsinc.org/) which mission is to facilitate the rescue and distribution of healthy, fresh food that would otherwise be discarded.
Specifically, they pick up fresh food that would otherwise be thrown away from grocery stores, produce wholesalers, farms and farmers markets, and distribute it to community non-profits that feed Greater Boston’s hungry.

### Data

* Node data
* Distance matrix

I had access to the operational data of *Lovin' Spoonful* but cannot display it online because I signed a NDA. The data provided by the organization is used as node data.
Further I developed a script to generate a distance matrix between each nodes (distance in second). This script is available in the repository.

### Objective
My goal was to find the best routes for the drivers of *Lovin' Spoonful*, in terms of the quantity of food rescued in a day.
I worked on multiple models to optimize the overall route schedule of *Lovin' Spoonful*.
The Travelling Salesman model aims specifically at finding the best route given a predetermined set of nodes.

### Asumption
The nature of the Travelling salesman model requires to have selected the nodes the driver will go through during his or her journey. The travelling salesman model alone is not helpful in determining which nodes should be included in the route and which should not.
It is possible to perform multiple iterations of the TSP on different sets of nodes and to find an optimal result put this approach would be rather inefficient.

In [69]:
access_path = '/home/eolus/Dropbox/610 Project Data'

# Libraries
import csv
import pandas as pd
import itertools
import pickle
import numpy as np
import math

## Data

## Initialize nodes

First, we load in memory the information related to each location the driver has to go through during the journey.
* `node_id`: unique identifier for a node
* `node_location`: exact address in the format [street_address, street_name, city, zip]
* `expected_load`: supply (case: expected_load > 0) or demand (case: expected_load < 0)
* `start_time`: beginning time of the node time window in 24:00hrs format
* `end_time`: ending time of the node time window in 24:00hrs format

*`node_location` values are hidden because I signed a Confidentiality Agreement with Lovin' Spoonfuls.*

In [70]:
node_data_df = pd.read_csv(access_path+'/node_data/node_data_test.csv', \
            header = 0)
node_data_df['node_location'] = "CONFIDENTIAL"
node_data_df

,node_id,node_location,expected_load,start_time,end_time
0,Lot,CONFIDENTIAL,0,00:00,23:59
1,WFCH,CONFIDENTIAL,650,9:00,15:00
2,CHS,CONFIDENTIAL,-614,11:00,13:00
3,RDE,CONFIDENTIAL,137,8:00,15:00
4,WFWO,CONFIDENTIAL,532,9:00,13:00
5,DWP,CONFIDENTIAL,-127,10:00,13:00
6,WFM,CONFIDENTIAL,429,9:00,14:00
7,MET,CONFIDENTIAL,-187,9:00,14:00
8,FHCC,CONFIDENTIAL,-616,10:00,13:00
9,WFC,CONFIDENTIAL,374,10:00,15:00


In [71]:
import time

# Convert time value to count of seconds after 00:00
def time_to_seconds(timeNum):
    timeStr = str(timeNum)
    timeData = time.strptime(timeStr, "%H:%M")
    minutes = timeData.tm_min
    hours = timeData.tm_hour
    
    time_in_seconds = hours * 3600 + minutes * 60
    return(time_in_seconds)

# Convert count of second to time in 24h format
def seconds_to_time(num_seconds):
    hour_val = str(int(num_seconds/3600))
    minute_val = str(int( (num_seconds % 3600) / 60))
   
    time_res = '{adj_hour}{hour_val}:{adj_minute}{minute_val}'.format(hour_val = hour_val,
                             minute_val = minute_val,
                             adj_hour = '0' if len(hour_val) < 2 else '',
                             adj_minute = '0' if len(minute_val) < 2 else '')
    return(time_res)

### Convert time values from 24:00 format to count of seconds

In [72]:
node_data_convert_start_time = node_data_df.start_time.astype(str).apply(time_to_seconds)
node_data_convert_end_time = node_data_df.end_time.astype(str).apply(time_to_seconds)

node_data_df['start_time'] = node_data_convert_start_time
node_data_df['end_time'] = node_data_convert_end_time
node_data_df

,node_id,node_location,expected_load,start_time,end_time
0,Lot,CONFIDENTIAL,0,0,86340
1,WFCH,CONFIDENTIAL,650,32400,54000
2,CHS,CONFIDENTIAL,-614,39600,46800
3,RDE,CONFIDENTIAL,137,28800,54000
4,WFWO,CONFIDENTIAL,532,32400,46800
5,DWP,CONFIDENTIAL,-127,36000,46800
6,WFM,CONFIDENTIAL,429,32400,50400
7,MET,CONFIDENTIAL,-187,32400,50400
8,FHCC,CONFIDENTIAL,-616,36000,46800
9,WFC,CONFIDENTIAL,374,36000,54000


### Nodes as `namedtuples`

I store the node objects as namedtuples for readability and efficiency.
The node object looks like this:

    `Node( ID, load, start_time, end_time, loading_time )`

In [73]:
from collections import namedtuple
Node = namedtuple('Node', 'ID load start_time end_time loading_time')

NODES = []

for node in node_data_df.iterrows():
    new_node = Node(
        ID=node[1]['node_id'],
        load=node[1]['expected_load'],
        start_time=node[1]['start_time'],
        end_time=node[1]['end_time'],
        loading_time=NODE_LOADING_TIME
        )
    NODES.append(new_node)
    
NUM_NODES = len(NODES)

for node in NODES:
    print(node)

Node(ID='Lot', load=0, start_time=0, end_time=86340, loading_time=900)
Node(ID='WFCH', load=650, start_time=32400, end_time=54000, loading_time=900)
Node(ID='CHS', load=-614, start_time=39600, end_time=46800, loading_time=900)
Node(ID='RDE', load=137, start_time=28800, end_time=54000, loading_time=900)
Node(ID='WFWO', load=532, start_time=32400, end_time=46800, loading_time=900)
Node(ID='DWP', load=-127, start_time=36000, end_time=46800, loading_time=900)
Node(ID='WFM', load=429, start_time=32400, end_time=50400, loading_time=900)
Node(ID='MET', load=-187, start_time=32400, end_time=50400, loading_time=900)
Node(ID='FHCC', load=-616, start_time=36000, end_time=46800, loading_time=900)
Node(ID='WFC', load=374, start_time=36000, end_time=54000, loading_time=900)
Node(ID='HCS', load=235, start_time=36000, end_time=54000, loading_time=900)
Node(ID='TH', load=-178, start_time=32400, end_time=61200, loading_time=900)
Node(ID='PAR', load=-201, start_time=39600, end_time=72000, loading_time=90

### Assign the source node 2 distinct variables: `node_origin` and  `node_arrival`

I store my routes in sets which by definition can only contain distinct objects, I cannot store a route that contains a same node twice.
However, a Travelling salesman route is a loop so the end node is the source node.
Therefore I created two nodes `node_origin` and  `node_arrival` that refer to the same source node to be able to store the TSP routes sets of nodes.

In [74]:
node_origin = None
for node in NODES:
    if node.ID == 'Lot':
        node_origin = node
node_origin

node_arrival = None
for node in NODES:
    if node.ID == 'Lot':
        node_arrival = node

## Distance matrix

I used the script `time_matrix.py` (available in the GitHub repository) to generate a distance matrix between each node.
The following code simply load the distance matrix previously generated in memory.

In [75]:
time_matrix_array = pickle.load( open(access_path + '/time_matrix.pkl', 'rb') )

assert(time_matrix_array.shape[0] == time_matrix_array.shape[1]), \
    "Time matrix is not square: {columns} columns // {rows} rows" \
    .format(columns = time_matrix_array.shape[0], rows = time_matrix_array.shape[1])

distance_matrix_pretty = pd.DataFrame(data=time_matrix_array, \
                                      index=node_data_df['node_id'], \
                                      columns=node_data_df['node_id'])

distance_matrix_pretty

node_id,Lot,WFCH,CHS,RDE,WFWO,DWP,WFM,MET,FHCC,WFC,HCS,TH,PAR,CAR
node_id,,,,,,,,,,,,,,
Lot,0,505,694,810,995,1238,1291,1007,1008,505,278,323,342,263
WFCH,511,0,164,285,970,914,844,517,533,186,297,340,283,323
CHS,703,192,0,319,981,925,878,551,619,270,472,454,476,473
RDE,803,337,375,0,958,935,695,382,396,448,590,592,576,615
WFWO,994,991,997,969,0,453,931,1151,1219,1105,943,992,1028,917
DWP,1264,876,881,878,461,0,698,1074,1141,989,1102,1085,1113,1104
WFM,1269,854,892,694,924,698,0,421,480,965,1106,1053,1092,1073
MET,1005,539,577,388,1140,1092,419,0,77,650,792,793,778,817
FHCC,982,510,652,383,1223,1166,492,74,0,575,776,838,762,802


In [76]:
# Get time distance between two nodes
def get_time_distance(from_node, to_node):
    dist = distance_matrix_pretty.loc[from_node, to_node]
    return int(dist)

# Example
nodeA = 'CHS'
nodeB = 'CAR'
print('Driving time from {nodeA} to {nodeB} = {timeAB} s'\
      .format(nodeA = nodeA, nodeB= nodeB, timeAB = get_time_distance('CHS', 'CAR')))

Driving time from CHS to CAR = 473 s


## Operational parameters

* `START_OF_DAY`: Truck start time in seconds from 12:00AM
* `END_OF_DAY`: Time limit to complete the trip in seconds from 12:00AM
* `LOAD_LIMIT`: Maximum truck capacity in LBS (optional) 
* `NODE_LOADING_TIME`: Time spent at each node for loading/unloading in seconds

In [77]:
START_OF_DAY = 8 * 3600              # Driver takes the road at 8:00 AM
END_OF_DAY = START_OF_DAY + 9 * 3600 # Driver's day lasts 9hours
LOAD_LIMIT = 999999999               # Artificially Inf truck load capacity
NODE_LOADING_TIME = 15 * 60          # Driver spends 15min to load/unload at each node

## Memoization dictionnary

The dynamic programming approach uses the fact that there is a single optimal route for each possible set of nodes. Each key, value pair in the memoization dictionnary represents an optimal route associated with a set of nodes.

* **Key**: set of nodes that composes the optimal route, `frozenset`
* **Value**: `Optimal_path`, `namedtuple` object with attributes:
    - `path`: optimal route for the set of nodes in the key, stored as a `list
    - `time_tracker`: equals START_TIME + time to complete `path`, in seconds
    - `load_tracker`: equals load after visiting all of the nodes of `path`, in LBS

In [78]:
Optimal_path = namedtuple('Optimal_path', 'path time_tracker load_tracker')

# Initialization of the Optimal_path: (Lot) : ([Lot], 0, 0 )
initialization_path = Optimal_path(
    path = ['Lot'],
    time_tracker=START_OF_DAY,
    load_tracker=0
    )
optimal_paths = {frozenset(['Lot']) : initialization_path}

print('Initialized memoization dictionnary:\n\n{memo}'.format(memo = optimal_paths))

Initialized memoization dictionnary:

{frozenset({'Lot'}): Optimal_path(path=['Lot'], time_tracker=28800, load_tracker=0)}


#### Generate best possible new paths of length `x+1` based on existing best paths of length `x`

Inputs:
* `level`: int corresponding to the decision stage. For `level` == `x`, the function will expand the optimal routes of length `x`.
* `optimal_paths`: memoization dictionnary storing the optimal routes found in previous iterations.
* `load_limit`: (optional) stores the maximum truck load constraint
* `num_nodes`: (optional) stores the number of nodes, is used to detect when to close the loop

Based on the existing optimal paths of length `level` in the memoization dictionnary (`optimal_paths`), the function will find a new set of `optimal_path` of length `level` +1. Each of those new routes represent the optimal route for their set of `level`+1 nodes.
The new routes are stored in the memoization dictionnary.

In [79]:
def clear_level(level, optimal_paths, load_limit=LOAD_LIMIT, num_nodes = NUM_NODES):
    
    # Builds list of optimal paths where length == level
    current_level_node_sets = \
    [node_set for node_set in optimal_paths.keys() if len(node_set) == level]
    
    # Iterate through optimal paths where length == level
    for node_set in current_level_node_sets:
        route = optimal_paths[node_set]    
        
        if level == num_nodes: # Case: Last node: close the loop, exit routine
            close_loop(route)
            continue
        else:                  # Case: Evaluate remaining nodes to include in routes
            route_path = route.path
            ending_node_ID = route_path[-1]
            remaining_nodes = []     # Build list of next_nodes to evaluate
            for node in NODES:
                if node.ID not in route_path:
                    remaining_nodes.append(node)
                    
        for new_node in remaining_nodes: # Iterate through remaining nodes to include
            new_node_ID = new_node.ID 
            new_path = list(route_path) + [new_node_ID] # Path being evaluated
            new_node_set = frozenset(new_path) # set of nodes of the path being evaluated
            total_load_after_node = route.load_tracker + new_node.load # Load variables
            node_loading_time = new_node.loading_time # Time for loading/unloading at node
            time_to_next_node = get_time_distance(ending_node_ID, new_node_ID) # Time-Distance
            legacy_time = route.time_tracker # Time-Distance variables
            total_time_after_node = legacy_time + time_to_next_node + node_loading_time

            # load_check
            if (   ( total_load_after_node < 0 ) or ( total_load_after_node > load_limit )):
                continue
            #Time window exception check
            start_time = new_node.start_time # Time-Window variables for the node
            end_time = new_node.end_time # Time-Window variables for the node
            # total_time check
            if (total_time_after_node > end_time):
                continue
            elif(legacy_time + time_to_next_node < start_time):
                total_time_after_node = start_time + node_loading_time
            if new_node_set in optimal_paths.keys():
                time_to_beat = optimal_paths[new_node_set].time_tracker
                if total_time_after_node < time_to_beat:
                    optimal_paths[new_node_set] = Optimal_path( \
                                path=new_path, 
                                time_tracker=total_time_after_node, 
                                load_tracker=total_load_after_node)
            else:
                optimal_paths[new_node_set] = Optimal_path( \
                                path=new_path, 
                                time_tracker=total_time_after_node, 
                                load_tracker=total_load_after_node)

In [80]:
# Function to close the loop and come back to origin Lot
def close_loop(route, node_origin = node_origin):
    
    route_path = route.path
    ending_node_ID = route_path[-1]
    node_origin_ID = node_origin.ID
    
    new_path = list(route_path) + ['Lot_arrival']
    new_node_set = frozenset(new_path)
    
    time_to_next_node = get_time_distance(ending_node_ID, node_origin_ID)
    legacy_time = route.time_tracker
    total_time_after_node = legacy_time + time_to_next_node
    
    total_load_after_node = route.load_tracker
    
    
    optimal_paths[new_node_set] = Optimal_path( \
                                path=new_path, 
                                time_tracker=total_time_after_node, 
                                load_tracker=total_load_after_node)

## Execution

In [81]:
# Basic timer to measure execution performance
def time_start_clock():
    return(time.time())

def time_elapsed(start):
    time.clock()
    elapsed = time.time() - start
    return(round(elapsed,3))

In [82]:
import time

for level in range(1, NUM_NODES+1):
   
    print('\tStage: {level}...'.format(level = level))
    print('*********************************')
    
    start_timing = time_start_clock()  # Initialize time for current stage
    clear_level(level, optimal_paths)  # Extend current subpath and store optimal paths
    timing = time_elapsed(start_timing)# time performance on current stage
    
    # Keeps track of number of optimal path at current stage
    current_level_node_sets = \
    [node_set for node_set in optimal_paths.keys() if len(node_set) == level]
    
    print('Optimal paths found:\t[{num_paths}]'.format(num_paths = len(current_level_node_sets)))
    print('Time performance:\t{time} sec'.format(time=timing))
    print()

	Stage: 1...
*********************************
Optimal paths found:	[1]
Time performance:	0.006 sec

	Stage: 2...
*********************************
Optimal paths found:	[6]
Time performance:	0.041 sec

	Stage: 3...
*********************************
Optimal paths found:	[40]
Time performance:	0.191 sec

	Stage: 4...
*********************************
Optimal paths found:	[142]
Time performance:	0.611 sec

	Stage: 5...
*********************************
Optimal paths found:	[358]
Time performance:	1.421 sec

	Stage: 6...
*********************************
Optimal paths found:	[640]
Time performance:	2.607 sec

	Stage: 7...
*********************************
Optimal paths found:	[861]
Time performance:	2.782 sec

	Stage: 8...
*********************************
Optimal paths found:	[857]
Time performance:	2.313 sec

	Stage: 9...
*********************************
Optimal paths found:	[649]
Time performance:	1.472 sec

	Stage: 10...
*********************************
Optimal paths found:	[358]
Tim

### Display results

In [83]:
import operator

# Function to return top results
def filter_results(opt_paths, num_nodes = NUM_NODES, num_results = 10):
    
    full_paths = []
    top_results = []
    
    for opt_path, details in opt_paths.items():
        if len(details.path) < num_nodes+1:
            continue
        else:
            full_paths.append((details.path, details.time_tracker, details.load_tracker))
    
    if num_results is None or num_results > len(full_paths):
        num_results = len(full_paths)
    print('Retrieved {num_results} optimal routes!\n'.format(num_results = num_results))
    
    print('Sorting routes by earliest arrival time...')
    
    while num_results > 0:
        num_results -= 1
        top_route = list(map(max, zip(*full_paths)))
        top_results.append(top_route)
        full_paths = [path for path in full_paths if  path[0] != top_route[0]]
        
    return(top_results)

#### Print top results

In [84]:
top_results = filter_results(optimal_paths)

for result in top_results:
    path = result[0]
    time = result[1]
    load = result[2]
    
    
    arrival_time = seconds_to_time(time)
    
    print('ROUTE:\n{path}'.format(path=path) )
    print('ARRIVAL TIME: {arr_time}'.format(arr_time=arrival_time) )
    print('END LOAD: {load}'.format(load=load) )
    print()

Retrieved 1 optimal routes!

Sorting routes by earliest arrival time...
ROUTE:
['Lot', 'RDE', 'WFM', 'WFWO', 'DWP', 'WFC', 'WFCH', 'CHS', 'FHCC', 'MET', 'CAR', 'HCS', 'TH', 'PAR', 'Lot_arrival']
ARRIVAL TIME: 13:27
END LOAD: 0

